### Importing necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import random
from bs4 import BeautifulSoup
from lxml import html

### Importing Data into Data Frame and Cleaning Data

Wikisource for Data = https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
#Scraping Data from Wikipedia
Wikihtml=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(Wikihtml.content,"html.parser")

Code=soup.find('table')
df=pd.read_html(str(Code))

#Turning Data into the Dataframe
Toronto=pd.DataFrame(df[0])

Toronto.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [20]:
#Merging Neighborhoods with same Postal Code
TorontoDf=Toronto.groupby(['Postal Code', 'Borough']) ['Neighborhood'].apply (','.join)
Torontodf2=pd.DataFrame(TorontoDf)
TorontoNew=Torontodf2.reset_index()
TorontoNew.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [27]:
#Dropping cells from Borough column that are "Not Assigned"
df1=TorontoNew[TorontoNew['Borough'] =='Not assigned'].index
TorontoNew.drop(df1, inplace=True)
TorontoNew.head(20)

,Postal Code,Borough,Neighborhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
10,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Show no. of rows of Dataframe

In [28]:
TorontoNew.shape

(103, 3)

### Getting Coordinates of each Neighborhoods

In [30]:
!pip install geocoder

In [33]:
import geocoder

lat_lng_coords = None

while(lat_lng_coords is None):
    g=geocoder.google('{}, Toronto, Ontario'.format('M1G'))#TorontoNew['Postal Code']))
    lat_lng_coords=g.latlng
    
latitude= lat_lng_coords[0]
longitude= lat_lng_coords[1]

KeyboardInterrupt: 

In [36]:
# Using CSV file for loading coordinates.Its taking too long with geocoder.
Coords=pd.read_csv('https://cocl.us/Geospatial_data')
Coords.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [38]:
CoordsNew=pd.merge(TorontoNew,Coords)
CoordsNew.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Clustering and Plotting

#### Importing libraries

In [39]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import json # library to handle JSON files

from pandas.io.json import json_normalize # transform JSON file in to a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Import k-means from clustering stage
from sklearn.cluster import KMeans

from IPython.display import Image 
from IPython.core.display import HTML

# Installing folium
!conda install -c conda-forge folium=0.5.0 --yes

# Importing map rendering library
import folium


print('Libraries imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

### Visualize Toronto using folium

In [40]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [45]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(CoordsNew['Latitude'], CoordsNew['Longitude'], CoordsNew['Borough'], CoordsNew['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Choose Boroughs with "East York"

In [47]:
EastYork=CoordsNew[CoordsNew['Borough']=='East York'].reset_index (drop=True)
EastYork

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
1,M4C,East York,Woodbine Heights,43.695344,-79.318389
2,M4G,East York,Leaside,43.709060,-79.363452
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372
4,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106


### Getting Geographical Coordinates of East York

In [48]:
#Creating map of Scarborough using latitude and longitude values
map_EastYork= folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to the map
for lat, lng, label in zip(EastYork['Latitude'],EastYork['Longitude'],EastYork['Neighborhood']):
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_EastYork)
    
map_EastYork

### Use Foursquare to collect information on selected Borough

In [49]:
CLIENT_ID = 'DWNFSRIEKVN23G0IPQ1L0UZ0K1CYPQ53MBYPJEZGDQC4DGDY' # your Foursquare ID
CLIENT_SECRET = '5QOWXSGE03DOCY4G5HAXRAJEDSIST2PBOLGZMDFYQYPY0BBR' # your Foursquare Secret
VERSION = '20200623'
LIMIT = 30
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: DWNFSRIEKVN23G0IPQ1L0UZ0K1CYPQ53MBYPJEZGDQC4DGDY
CLIENT_SECRET:5QOWXSGE03DOCY4G5HAXRAJEDSIST2PBOLGZMDFYQYPY0BBR


#### Explore Neighborhoods in Dataframe

In [53]:
#get the neighborhood's name.
EastYork.loc [0,'Neighborhood']

'Parkview Hill, Woodbine Gardens'

In [54]:
#Get the neighborhood's latitude and longitude values:
neighborhood_latitude = EastYork.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = EastYork.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = EastYork.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkview Hill, Woodbine Gardens are 43.7063972, -79.309937.


### Get the top 100 venues in Parkview Hill, Woodbine Gardens within a 500 meter radius

In [55]:
LIMIT=100 #Limit no of venues returned by Foursquare API
radius=500 #define radius

#create URL
url=  'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=DWNFSRIEKVN23G0IPQ1L0UZ0K1CYPQ53MBYPJEZGDQC4DGDY&client_secret=5QOWXSGE03DOCY4G5HAXRAJEDSIST2PBOLGZMDFYQYPY0BBR&v=20200623&ll=43.7063972,-79.309937&radius=500&limit=100'

### Get JSON file

In [57]:
results= requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef21291c94979001b779fec'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': "O'Connor - Parkview",
  'headerFullLocation': "O'Connor - Parkview, Toronto",
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 43.7108972045, 'lng': -79.30372360313615},
   'sw': {'lat': 43.701897195499996, 'lng': -79.31615039686386}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b5a3842f964a52023b528e3',
       'name': 'Jawny Bakers',
       'location': {'address': "804 O'Connor Dr",
        'crossStreet': 'St Clair E',
        'lat': 43.705782646822,
        'lng': -79.31291304477831,
        'labeledLatLngs':

In [58]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Cleaning JSON file and structuring to pandas dataframe

In [60]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jawny Bakers,Gastropub,43.705783,-79.312913
1,East York Gymnastics,Gym / Fitness Center,43.710654,-79.309279
2,TD Canada Trust,Bank,43.705740,-79.312270
3,Shoppers Drug Mart,Pharmacy,43.705933,-79.312825
4,Pizza Pizza,Pizza Place,43.705159,-79.313130


In [63]:
print('{} venues of Parkview Hill are returned by Foursquare'.format(nearby_venues.shape[0]))

10 venues of Parkview Hill are returned by Foursquare


### Explore Neighborhoods in East York

In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [66]:
eastyork_venues = getNearbyVenues(names=EastYork['Neighborhood'],
                                   latitudes=EastYork['Latitude'],
                                   longitudes=EastYork['Longitude']
                                  )

Parkview Hill, Woodbine Gardens
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)


#### Check size of resulting Dataframe

In [67]:
print(eastyork_venues.shape)
eastyork_venues.head()

(74, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Jawny Bakers,43.705783,-79.312913,Gastropub
1,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,East York Gymnastics,43.710654,-79.309279,Gym / Fitness Center
2,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,TD Canada Trust,43.705740,-79.312270,Bank
3,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Shoppers Drug Mart,43.705933,-79.312825,Pharmacy
4,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Pizza Pizza,43.705159,-79.313130,Pizza Place


In [69]:
#Check how many venues returned for each neighborhood
eastyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"East Toronto, Broadview North (Old East York)",3,3,3,3,3,3
Leaside,33,33,33,33,33,33
"Parkview Hill, Woodbine Gardens",10,10,10,10,10,10
Thorncliffe Park,22,22,22,22,22,22
Woodbine Heights,6,6,6,6,6,6


#### Find out how many unique categories can be curated from all the returned venues

In [70]:
print('There are {} uniques categories.'.format(len(eastyork_venues['Venue Category'].unique())))

There are 45 uniques categories.


### Analyze each Neighborhood

In [71]:
# one hot encoding
eastyork_onehot = pd.get_dummies(eastyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eastyork_onehot['Neighborhood'] = eastyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [eastyork_onehot.columns[-1]] + list(eastyork_onehot.columns[:-1])
manhattan_onehot = eastyork_onehot[fixed_columns]

eastyork_onehot.head()

,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Coffee Shop,Convenience Store,...,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Warehouse Store,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Parkview Hill, Woodbine Gardens"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Parkview Hill, Woodbine Gardens"
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Parkview Hill, Woodbine Gardens"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Parkview Hill, Woodbine Gardens"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Parkview Hill, Woodbine Gardens"


In [72]:
# Size of new dataframe
eastyork_onehot.shape

(74, 46)

###  Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category and confirm new size

In [73]:
eastyork_grouped = eastyork_onehot.groupby('Neighborhood').mean().reset_index()
eastyork_grouped

,Neighborhood,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Coffee Shop,...,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Warehouse Store,Yoga Studio
0,"East Toronto, Broadview North (Old East York)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Leaside,0.000000,0.030303,0.060606,0.030303,0.030303,0.030303,0.030303,0.060606,0.090909,...,0.030303,0.030303,0.000000,0.030303,0.060606,0.030303,0.030303,0.030303,0.000000,0.000000
2,"Parkview Hill, Woodbine Gardens",0.100000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Thorncliffe Park,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,...,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.045455,0.045455
4,Woodbine Heights,0.166667,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [74]:
eastyork_grouped.shape

(5, 46)

In [76]:
num_top_venues = 5

for hood in eastyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = eastyork_grouped[eastyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----East Toronto, Broadview North (Old East York)----
                venue  freq
0                Park  0.67
1   Convenience Store  0.33
2  Athletics & Sports  0.00
3          Restaurant  0.00
4   Indian Restaurant  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1                    Bank  0.06
2            Burger Joint  0.06
3     Sporting Goods Shop  0.06
4  Furniture / Home Store  0.06


----Parkview Hill, Woodbine Gardens----
                venue  freq
0         Pizza Place   0.2
1  Athletics & Sports   0.1
2        Intersection   0.1
3           Gastropub   0.1
4           Pet Store   0.1


----Thorncliffe Park----
                  venue  freq
0        Sandwich Place  0.09
1     Indian Restaurant  0.09
2                   Gym  0.05
3  Fast Food Restaurant  0.05
4       Warehouse Store  0.05


----Woodbine Heights----
                venue  freq
0  Athletics & Sports  0.17
1          Beer Store  0.17
2                Park  0.17
3        Ska

### Convert to pandas dataframe

In [77]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = eastyork_grouped['Neighborhood']

for ind in np.arange(eastyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eastyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"East Toronto, Broadview North (Old East York)",Park,Convenience Store,Yoga Studio,Department Store,Gas Station,Furniture / Home Store,Food & Drink Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
1,Leaside,Coffee Shop,Furniture / Home Store,Sporting Goods Shop,Burger Joint,Bank,Gym,Electronics Store,Fish & Chips Shop,Food & Drink Shop,Dessert Shop
2,"Parkview Hill, Woodbine Gardens",Pizza Place,Athletics & Sports,Gym / Fitness Center,Fast Food Restaurant,Intersection,Pet Store,Pharmacy,Gastropub,Bank,Burger Joint
3,Thorncliffe Park,Sandwich Place,Indian Restaurant,Yoga Studio,Middle Eastern Restaurant,Bank,Burger Joint,Coffee Shop,Discount Store,Fast Food Restaurant,Gas Station
4,Woodbine Heights,Athletics & Sports,Skating Rink,Park,Curling Ice,Pharmacy,Beer Store,Bike Shop,Department Store,Furniture / Home Store,Food & Drink Shop


In [79]:
neighborhoods_venues_sorted.shape

(5, 11)

## Cluster Neighborhoods

###### Use K-means to cluster neighborhood into 5 clusters

In [88]:
# set number of clusters
kclusters = 5

eastyork_grouped_clustering = eastyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(eastyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 4, 1, 0], dtype=int32)

In [111]:
# add clustering labels
neighborhoods_venues_sorted.insert(0,'Cluster1', kmeans.labels_)

eastyork_merged = EastYork

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
eastyork_merged = eastyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

eastyork_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster1,Cluster,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,4,4,4,4,Pizza Place,Athletics & Sports,Gym / Fitness Center,Fast Food Restaurant,Intersection,Pet Store,Pharmacy,Gastropub,Bank,Burger Joint
1,M4C,East York,Woodbine Heights,43.695344,-79.318389,0,0,0,0,Athletics & Sports,Skating Rink,Park,Curling Ice,Pharmacy,Beer Store,Bike Shop,Department Store,Furniture / Home Store,Food & Drink Shop
2,M4G,East York,Leaside,43.709060,-79.363452,3,3,3,3,Coffee Shop,Furniture / Home Store,Sporting Goods Shop,Burger Joint,Bank,Gym,Electronics Store,Fish & Chips Shop,Food & Drink Shop,Dessert Shop
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372,1,1,1,1,Sandwich Place,Indian Restaurant,Yoga Studio,Middle Eastern Restaurant,Bank,Burger Joint,Coffee Shop,Discount Store,Fast Food Restaurant,Gas Station
4,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,2,2,2,2,Park,Convenience Store,Yoga Studio,Department Store,Gas Station,Furniture / Home Store,Food & Drink Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store


In [97]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eastyork_merged['Latitude'], eastyork_merged['Longitude'], eastyork_merged['Neighborhood'], eastyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining Clusters

In [112]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 0, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster1,Cluster,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East York,0,0,0,0,Athletics & Sports,Skating Rink,Park,Curling Ice,Pharmacy,Beer Store,Bike Shop,Department Store,Furniture / Home Store,Food & Drink Shop


In [113]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 1, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster1,Cluster,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East York,1,1,1,1,Sandwich Place,Indian Restaurant,Yoga Studio,Middle Eastern Restaurant,Bank,Burger Joint,Coffee Shop,Discount Store,Fast Food Restaurant,Gas Station


In [114]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 2, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster1,Cluster,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East York,2,2,2,2,Park,Convenience Store,Yoga Studio,Department Store,Gas Station,Furniture / Home Store,Food & Drink Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store


In [116]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 3, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster1,Cluster,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East York,3,3,3,3,Coffee Shop,Furniture / Home Store,Sporting Goods Shop,Burger Joint,Bank,Gym,Electronics Store,Fish & Chips Shop,Food & Drink Shop,Dessert Shop
